# Question  7


---

## Train and fine-tune a Decision Tree for the moons dataset.

1. Generate a moons dataset using make_moons(n_samples=10000, noise=0.4).
2. Split it into a training set and a test set using train_test_split().
3. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.
4. Train it on the full training set using these hyperparameters, and measure your model’s
performance on the test set. You should get roughly 85% to 87% accuracy.



Step 1 - Generating dataset 


In [68]:
import warnings
warnings.filterwarnings("ignore")

In [69]:
from sklearn.datasets import make_moons
X,y = make_moons(n_samples=10000,shuffle=True,noise=0.4)

In [70]:
import numpy as np
np.unique(y,return_counts=True)

(array([0, 1]), array([5000, 5000]))

We can say that dataset is balanced, so it is perfect for decision tree

Step 2 - train test split

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Step 3 - Finding best hyperparamters value for DT

In [72]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
parameters = {'max_leaf_nodes':[1,5,10,20,50]}
clf = GridSearchCV(DT,parameters)
clf.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_leaf_nodes': [1, 5, 10, 20, 50]},
           

In [73]:
clf.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=20,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

Hence max_leaf_node = 20 is best Hyperparameter for this problem

Step 4 - Training model using best hyperparameter and testing model performance

In [74]:
from sklearn.metrics import accuracy_score
DT = DecisionTreeClassifier(max_leaf_nodes = 20)
DT.fit(X_train,y_train)
y_pred = DT.predict(X_test)
print("The accuracy of the model",round(accuracy_score(y_test,y_pred),4))

The accuracy of the model 0.8618


# Question  8


---

## Grow a forest.
1. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100
instances selected randomly. Hint: you can use Scikit-Learn’s ShuffleSplit class for this.
2. Train one Decision Tree on each subset, using the best hyperparameter values found above.
Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets,
these Decision Trees will likely perform worse than the first Decision Tree, achieving only
about 80% accuracy.
3. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision
Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for
this). This gives you majority-vote predictions over the test set.
4. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your
first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest
classifier!



Step 1 - Small subset of Dataset

In [75]:
from sklearn.model_selection import ShuffleSplit
split_dataset = ShuffleSplit(n_splits=1000,train_size=100)
subset_train_dataset = []
for train_indices,test_indices in split_dataset.split(X_train):
   subset_train_dataset.append([X_train[train_indices],y_train[train_indices]])

Step 2 - 
1. Traning 1000 decision tree on train subsets and evulating them on test set

In [76]:
model_list = [0 for i in range(1000)]
model_index = 0
for dataset in subset_train_dataset:
  X_train,y_train = dataset
  model_list[model_index] = DecisionTreeClassifier(max_leaf_nodes = 20)
  model_list[model_index].fit(X_train,y_train)
  model_index +=1

In [77]:
Accuracy_model = []
for model in model_list:
  y_pred = model.predict(X_test)
  Accuracy_model.append(accuracy_score(y_test,y_pred))

In [78]:
print("The average accuracy of all the models on the test dataset is {}".format(round(sum(Accuracy_model)/len(Accuracy_model),4)))

The average accuracy of all the models on the test dataset is 0.7938


We can see performance is not as good as first model

Step 3 - Doing majority vote prediction of all models

In [79]:
from scipy.stats import mode
model_prediction = []
for model in model_list:
  model_prediction.append(model.predict(X_test))

In [80]:
majority_vote = mode(model_prediction)

Step 4 - prediction on test set (Random Forest)

In [81]:
print("The accuracy of majority vote on the test dataset is {}".format(round(accuracy_score(y_test,majority_vote[0][0]),4)))

The accuracy of majority vote on the test dataset is 0.867


Hence we can see that performance of the model is increased